# week14 创建你的 AI 应用


## AI 与现实世界的交互
### 看不见的开关
AI & IoT

<!--按下看不见的开关，打开电灯

如果食指(id 8)和拇指(id 4)足够接近就触发开关-->

#### 使用 Python 控制物联网设备
使用 Python 和英荔龙眼盒交互

In [1]:
import requests
import functools
import threading
import time

thingtalk_url = f'https://2ecf620.wormhole.longan.link'

def get_token():
    # 获取访问凭证
    url = f"{thingtalk_url}/auth/jwt/login"
    data = {
        "username": "admin",
        "password": "20130530"
    }
    res = requests.post(url=url, data=data)
    if res.status_code != 200:
        raise Exception(res.status_code)
    token = res.json()["access_token"]
    return token

token = get_token()

# print(f"token: {token}")

def set_properties(thing_id, prop, data):
    # 设置设备状态
    url = f"{thingtalk_url}/things/{thing_id}/properties/{prop}"
    headers = {"Authorization": f"Bearer {token}"}
    res = requests.put(url=url, headers=headers, json=data)
    # print(res.text)
    if res.status_code != 200:
        raise Exception(res.text)

In [2]:
'''
1号 0x00124b00210b7ad9
2号 0x00124b00210b7c9c
3号 0x00124b00210b7124
4号 0x00124b00210b7aaf
'''
device_id = '0x00124b00210b7ad9'
set_properties(device_id,'state', data={"state":"OFF"})

<Thread(1641542801.231918, started daemon 123145520177152)>

In [ ]:
# 同时控制多个设备 并发
import concurrent.futures
# def control_2_devices_immediately(device1,device2):
executor = ThreadPoolExecutor(max_workers=2)
# 同时关闭 2 个设备 0x00124b00210b7ad9 和 0x00124b00210b7a3d
executor.submit(set_properties, '0x00124b00210b7ad9', 'state', {"state":"OFF"}) # 不等待
executor.submit(set_properties, '0x00124b00210b7a3d', 'state', {"state":"OFF"})

### 看不见的开关(可编程空间版本)

In [ ]:
import cv2
import mediapipe
import pyautogui
import time

mp_drawing = mediapipe.solutions.drawing_utils
mp_drawing_styles = mediapipe.solutions.drawing_styles
mp_hands = mediapipe.solutions.hands


# 获取摄像头
cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0,
                    min_detection_confidence=0.5,  # confidence: 置信度，信心
                    min_tracking_confidence=0.5) as hands:
    
    light_state = False
    last_time = time.time()
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # writeable 用于性能优化
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 识别结果，手部信息都包含在 results 里
        results = hands.process(image)

        # 在图像上绘制额外信息.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # landmark: 地标 特征点
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                for id , lm in enumerate(hand_landmarks.landmark):
                    h , w , c = image.shape 
                    # get the X,Y of a finger
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    if id == 4:
                        # 拇指坐标
                        coordinates4 =  (cx, cy)
                    if id == 8:
                        # 食指坐标
                        coordinates8 =  (cx, cy)
            
            try:
                # 两个食指之间的距离
                distance = abs(coordinates8[0] - coordinates4[0]) + abs(coordinates8[1] - coordinates4[1])
                # 在食指显示与拇指的距离信息
                cv2.putText(image, str(distance), (coordinates8[0], coordinates8[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2, cv2.LINE_AA)
                if distance < 50:
                        # 如果两根手指靠近(捏一下🤏)
                        if  time.time() - last_time > 1:
                            last_time = time.time()
                            # 改变灯的状态
                            light_state = not light_state
                            # cv2.circle(image , (200, 200), 50, (0, 165, 255), -1)
                            if light_state:
                                set_properties(device_id,'state', data={"state":"ON"})
                                # todo 控制多盏灯
                            else:
                                set_properties(device_id,'state', data={"state":"OFF"})
                                
            except Exception as e:
                print(e)
                
            # 根据灯的状态，显示出虚拟的灯
            if light_state:
                cv2.circle(image , (200, 200), 50, (0, 165, 255), -1)
            else:
                cv2.circle(image , (200, 200), 50, (128, 128, 128), -1)

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', image)  # cv2.flip(image, 1)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()